In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime
from time import sleep

In [ ]:
short_data = pd.read_csv("most-experiments/media/ultimate_contra.csv", sep=",")

In [ ]:
short_data["Date"] = short_data["Date"].astype("datetime64[ns]")

In [ ]:
short_data.groupby("Ticker").first()

In [ ]:
short_data.groupby("Ticker").first().reset_index().values[:, :2]

In [ ]:
prices = {}
for ticker, start_date in short_data.groupby("Ticker").first().reset_index().values[:, :2]:
    data = yf.download(ticker, start=start_date, end=datetime.now(), interval="1wk")
    prices[ticker] = data
    sleep(1)

In [ ]:
results = {}
for ticker in short_data.Ticker.unique():
    # print(ticker)
    short_data_filter = short_data[short_data["Ticker"] == ticker]
    ticker_price = prices[ticker]["Close"] / prices[ticker]["Close"].shift(1)
    ticker_price[ticker] = ticker_price[ticker].shift(-1)
    ticker_price["invest"] = 0
    for d in short_data_filter.values[:, :1]:
        # print(d)
        d = d[0]
        ticker_price.loc[ticker_price[ticker_price.index >= d].index[0], "invest"] = 1
        # display(ticker_price)
        ticker_price["cum_sum"] = ticker_price["invest"].cumsum()
        inv = 0
        inv_a = []
        for row in ticker_price.iterrows():
            inv += row[1]["invest"]
            inv *= row[1][ticker] if pd.notna(row[1][ticker]) else 1
            inv_a.append(inv)
        ticker_price["return"] = inv_a
        ticker_price["Contrarian"] = ((ticker_price["return"] / ticker_price["cum_sum"]) -1) * 100
        results[ticker] = ticker_price

In [ ]:
import plotly.express as px

In [ ]:
# Combine all DataFrames into one with a 'strategy' column
combined_df = pd.concat(
    [df.assign(strategy=t) for t, df in results.items()]
)

# Plot all lines in one figure
fig = px.line(combined_df, x=combined_df.index, y="Contrarian", color="strategy", title="Return Percentage by Strategy", template="plotly_dark")
fig.show()

In [ ]:
combined_df

In [ ]:
spy = data = yf.download('SPY', start='2024-03-11', end='2025-04-23', interval="1wk")

In [ ]:
spy["gain"] = spy["Close"]["SPY"] / spy["Close"]["SPY"].shift(1)

In [ ]:
inv_spy = 1
inv_spy_a = []
for g in spy["gain"].values: 
    inv_spy *= g if pd.notna(g) else 1
    inv_spy_a.append(inv_spy)

In [ ]:
plot_compare = combined_df.groupby("Date")["Contrarian"].mean().reset_index()
plot_compare["SPY"] = (np.array(inv_spy_a) - 1) * 100 

In [ ]:
px.line(plot_compare, x="Date", y=["SPY", "Contrarian"], template="plotly_dark", labels={"value": "Percentage Gain", "variable": "Strategy"})
# px.scatter(plot_compare, x="Date", y=["SPY", "Contrarian"], template="plotly_dark", labels={"value": "Percentage Gain", "variable": "Strategy"})

---

In [ ]:
combined_df.groupby("strategy").last().reset_index().sort_values("Contrarian", ascending=False)

In [ ]:
port = combined_df.groupby("strategy").last().reset_index()
c = dict(zip(port["strategy"].unique(), px.colors.qualitative.Dark24))
fig = px.pie(port, values="cum_sum", names="strategy", template="plotly_dark", color="strategy", color_discrete_map=c)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(port, values="return", names="strategy", template="plotly_dark", color="strategy", color_discrete_map=c)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
port = combined_df.groupby("strategy").last().reset_index()
port["initial_perc"] = port["cum_sum"] / port["cum_sum"].sum()
port["port_perc"] = port["return"] / port["return"].sum()

In [ ]:
port_a = port[["strategy", "initial_perc"]].rename(columns={"initial_perc": "perc"})
port_a["a"] = "initial"

port_b = port[["strategy", "port_perc"]].rename(columns={"port_perc": "perc"})
port_b["a"] = "return"

port = pd.concat([port_a, port_b])
port

In [ ]:
fig = px.area(port, x="a", y="perc", color="strategy", template="plotly_dark")
#fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

----

In [ ]:
results_cut_looser = {}
for ticker in short_data.Ticker.unique():
    # print(ticker)
    short_data_filter = short_data[short_data["Ticker"] == ticker]
    ticker_price = prices[ticker]["Close"] / prices[ticker]["Close"].shift(1)
    ticker_price[ticker] = ticker_price[ticker].shift(-1)
    ticker_price["invest"] = 0
    for d in short_data_filter.values[:, :1]:
        # print(d)
        d = d[0]
        ticker_price.loc[ticker_price[ticker_price.index >= d].index[0], "invest"] = 1
        # display(ticker_price)
        ticker_price["cum_sum"] = ticker_price["invest"].cumsum()
        inv = 0
        inv_a = []
        for row in ticker_price.iterrows():
            inv += row[1]["invest"]
            inv *= row[1][ticker] if pd.notna(row[1][ticker]) else 1
            inv_a.append(inv)
        
        ticker_price["return"] = inv_a
        ticker_price["return_perc"] = ((ticker_price["return"] / ticker_price["cum_sum"]) -1) * 100
        stop_loss = ticker_price["return_perc"].values
        for i, a in enumerate(stop_loss):
            if a < -20:
                stop_loss[i:] = [a] * (len(stop_loss) - i)
                break
        ticker_price["return_perc"] = stop_loss
        results_cut_looser[ticker] = ticker_price

In [ ]:
results_cut_looser["MRNA"]

In [ ]:
# Combine all DataFrames into one with a 'strategy' column
combined_df = pd.concat(
    [df.assign(strategy=t) for t, df in results_cut_looser.items()]
)

# Plot all lines in one figure
fig = px.line(combined_df, x=combined_df.index, y="return_perc", color="strategy", title="Return Percentage by Strategy")
fig.show()

In [ ]:
plot_compare = combined_df.groupby("Date")["return_perc"].mean().reset_index()
plot_compare["SPY"] = (np.array(inv_spy_a) - 1) * 100 

In [ ]:
px.line(plot_compare, x="Date", y=["SPY", "return_perc"])